In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
from cantine import get_df, get_results

In [ ]:
get_results(base)

In [ ]:
df = get_df()

In [ ]:
resume = (
    df.groupby(["QF", "N° FAM", "N° PER"])
    .NOMBRE.sum()
    .reset_index()
    .sort_values("QF", ascending=False)
)

resume[resume.QF < 90000].QF.hist(bins=50)

In [ ]:
resume[resume.QF < 1000].QF.hist(bins=26)

In [ ]:
resume[resume.QF < 1000].QF.hist(bins=26)

In [ ]:
resume[resume.QF < 90000]

In [ ]:
resume.QF.describe()

In [ ]:
resume.NOMBRE.describe()

In [ ]:
resume.NOMBRE.hist(bins=128, cumulative=True, density=True)

In [ ]:
df.NOMBRE.sum() / 240

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df[["QF", "NOMBRE"]].groupby(by="QF").sum())

In [ ]:
df[df.QF != df.QF.max()].sort_values(by="QF", ascending=False)

In [ ]:
df[df.QF < df.QF.max()].QF.describe(
    percentiles=[0.01, 0.1, 0.25, 0.50, 0.75, 0.9, 0.99]
)

In [ ]:
df[df.QF < df.QF.max()].QF.hist(bins=500, cumulative=True, density=True)

In [ ]:
df[df.QF < 5000].QF.hist(bins=500, cumulative=True, density=True)

In [ ]:
df[df.QF < 2000].QF.hist(bins=40)

In [ ]:
df[df.QF < 1000].QF.hist(bins=50)

In [ ]:
r = get_results(base)
r

In [ ]:
r.sum()